##PotterGPT (Kind of)

More like PotterGPRNNs

Use textual data from the book The Philosopher's Stone to train a generative RNN which can complete sentences for you.


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import random
import numpy as np

### Read the data

In [ ]:
!git clone https://github.com/Anweshbyte/whiteboard.git

Cloning into 'whiteboard'...
remote: Enumerating objects: 173, done.
remote: Total 173 (delta 0), reused 0 (delta 0), pack-reused 173
Receiving objects: 100% (173/173), 27.92 MiB | 16.68 MiB/s, done.
Resolving deltas: 100% (29/29), done.


In [ ]:
# Load and preprocess the text data
text = open("/content/whiteboard/nbviewer/notebooks/data/harrypotter/Book 1 - The Philosopher's Stone.txt", 'r').read()

##Training and Labels
We would like to split our input data into Training Data and label.
How do we go about it?

Say we have a sentence:- I love the Transformers.

The training data will look like: "I love the"
and the corresponding label will look like: "I love the T"
That is, the model must predict the next letter in the sequence.

The next training sample will be shifted to the right by one.

In [ ]:
sample_text = "The Transformer Nostradamus"
sequence_length = 10
sequences = []
next_chars = []
for i in range(len(sample_text) - sequence_length):
    sequences.append(sample_text[i:i + sequence_length])
    next_chars.append(sample_text[i+sequence_length]) #Predict the next character in the sequence

print(sequences)
print(next_chars)

['The Transf', 'he Transfo', 'e Transfor', ' Transform', 'Transforme', 'ransformer', 'ansformer ', 'nsformer N', 'sformer No', 'former Nos', 'ormer Nost', 'rmer Nostr', 'mer Nostra', 'er Nostrad', 'r Nostrada', ' Nostradam', 'Nostradamu']
['o', 'r', 'm', 'e', 'r', ' ', 'N', 'o', 's', 't', 'r', 'a', 'd', 'a', 'm', 'u', 's']


In [ ]:
# Create input-output pairs

sequence_length = 50
sequences = []
next_chars = []
for i in range(len(text) - sequence_length):
    sequences.append(text[i:i + sequence_length])    # Apply the previous operation on the entire text
    next_chars.append(text[i+sequence_length])   # Apply the previous operation on the entire text

In [ ]:
# Vectorize the data
chars = np.unique(sorted(text)) # Create a list of unique characters present in the text data
char_indices = {char: index for index, char in enumerate(chars)}
num_chars = len(chars)

x = np.zeros((len(sequences), sequence_length, num_chars), dtype=np.bool)
y = np.zeros(shape=(len(sequences),num_chars)) # Initialize a 2D numpy array with dimensions (len(sequences), num_chars). It will be used to store the target data for the neural network. each element in y is initialized to zero.


for i, sequence in enumerate(sequences):
    for t, char in enumerate(sequence):
        x[i, t, char_indices[char]] = 1  # Set the value at the appropriate position in the x array to 1
    y[i, char_indices[next_chars[i]]] = 1

<ipython-input-6-4b0450f7087f>:6: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  x = np.zeros((len(sequences), sequence_length, num_chars), dtype=np.bool)


In [ ]:
# Build the RNN model
model = Sequential() # define a sequential model

model.add(LSTM(128, input_shape=(sequence_length, num_chars))) # Add LSTM layer with 128 features and proper input shape
model.add(Dense(num_chars, activation='softmax'))

In [ ]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [ ]:
# Train the model
epochs = 5 # With the small number of training epochs, the model won't learn to form coherent sentences. The easiest thing you can do to improve the results is to train it for longer (try EPOCHS = 30).
batch_size = 64
model.fit(x, y, epochs=epochs, batch_size = batch_size, verbose=2) # Using model.fit() train the model for a fixed number of epochs (dataset iterations).

Epoch 1/5
7413/7413 - 44s - loss: 2.1921 - 44s/epoch - 6ms/step
Epoch 2/5
7413/7413 - 35s - loss: 1.7404 - 35s/epoch - 5ms/step
Epoch 3/5
7413/7413 - 34s - loss: 1.5883 - 34s/epoch - 5ms/step
Epoch 4/5
7413/7413 - 35s - loss: 1.4952 - 35s/epoch - 5ms/step
Epoch 5/5
7413/7413 - 35s - loss: 1.4324 - 35s/epoch - 5ms/step


In [ ]:
# Generate text
start_index = random.randint(0, len(text)-sequence_length - 1) # Choose random indices between 0(inclusive) and len(text) - sequence_length -1 (exclusive)
seed_sequence = text[start_index:start_index + sequence_length]

We have trained it for only 1 epoch, so the performance of the model is BAD. Try training it for 10 epochs, and then 50 epochs, and you will see a better performance.

In [ ]:
generated_text = "The letter from Dumbledore said that " #You can play with this!
for i in range(200):
    x_pred = np.zeros((1, sequence_length, num_chars))
    for t, char in enumerate(seed_sequence):
        x_pred[0, t, char_indices[char]] = 1
    predicted_probs = model.predict(x_pred, verbose=0)[0]
    next_char_index = np.random.choice(range(num_chars), p=predicted_probs)
    next_char = chars[next_char_index] # Retrieve the character corresponding to the sampled index from the `chars` list and store it in the `next_char` variable.
    generated_text += next_char # Append the `next_char` to the `generated_text`
    seed_sequence = seed_sequence[1:] + next_char #  Update the `seed_sequence` by removing the first character and adding the `next_char` at the end. This step prepares the updated `seed_sequence` for the next iteration.

print(generated_text)

The letter from Dumbledore said that Gryaking lin here out aworting whippars 
stands out of drive a school was jot over the wly my 
finch atsents up one for again. 

“Sove easwaggin.” 

“Wht’ latelon?” said Harry. 

The smalloppear dark 
